In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pip install youtube-transcript-api


In [ ]:
pip install transformers


In [ ]:
import youtube_transcript_api
from youtube_transcript_api import YouTubeTranscriptApi
import nltk
from nltk.corpus import stopwords
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
import numpy as np
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration, pipeline

# Ensure necessary nltk resources are downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Get subtitles
link = "https://www.youtube.com/watch?v=6uUblznfrsk"
unique_id = link.split("=")[-1]
sub = YouTubeTranscriptApi.get_transcript(unique_id)
subtitle = " ".join([x['text'] for x in sub])

# Summarization using TF-IDF Vectorizer
# Summarization using TF-IDF Vectorizer
subtitle = subtitle.replace("\n", "")
sentences = sent_tokenize(subtitle)
organized_sent = {k: v for v, k in enumerate(sentences)}

# Adjusted TF-IDF Vectorizer parameters
tf_idf = TfidfVectorizer(min_df=1,  # Set min_df to 1 to ensure it works with fewer sentences
                        strip_accents='unicode',
                        max_features=None,
                        lowercase=True,
                        token_pattern=r'\w{1,}',
                        ngram_range=(1, 3),
                        use_idf=1,
                        smooth_idf=1,
                        sublinear_tf=1,
                        stop_words='english')

# Fit and transform sentences
sentence_vectors = tf_idf.fit_transform(sentences)
sent_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

N = 3
top_n_sentences = [sentences[index] for index in np.argsort(sent_scores, axis=0)[::-1][:N]]

mapped_sentences = [(sentence, organized_sent[sentence]) for sentence in top_n_sentences]
mapped_sentences = sorted(mapped_sentences, key=lambda x: x[1])
ordered_sentences = [element[0] for element in mapped_sentences]
summary_tf_idf = " ".join(ordered_sentences)


# Summarization using BART
try:
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

    input_tensor = tokenizer.encode(subtitle, return_tensors="pt", max_length=512, truncation=True)
    outputs_tensor = model.generate(input_tensor, max_length=160, min_length=120, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary_bart = tokenizer.decode(outputs_tensor[0], skip_special_tokens=True)
except Exception as e:
    summary_bart = str(e)

# Summarization using transformers' summarization pipeline
try:
    summarizer = pipeline('summarization')
    summary_pipeline = summarizer(subtitle, max_length=180, min_length=30, truncation=True)
except Exception as e:
    summary_pipeline = str(e)

# Output summaries
print("TF-IDF Summary:")
print(summary_tf_idf)
print("\nBART Summary:")
print(summary_bart)
print("\nPipeline Summary:")
print(summary_pipeline)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

TF-IDF Summary:
if everyone's problems were like laundry hanging out to dry in an open field and you had the opportunity to go out into that field and exchange your laundry your problems in other words with anyone else's you would still choose your own problems I'm I'm not the usual suspect class day speaker for sure I come from another profession the often undiplomatic world of Commerce I come from halfway the world I'm not an alumnist but when I asked Dean kite she said the reasons for which they invited me were in fact all of the above but I'm not here to try and dispense any unique wisdom as the proverbial school girl wrote Socrates was a wise Greek philosopher and he went around dispensing gratuitous advice and they poisoned him so I deeply fear that fate so what I will do is to just throw some ideas at you based on my own life experience and my own world that I hope will be relevant to any profession and to every life now picture if you will the world in 1946 when my fathers stoo